In [5]:
import requests
import numpy as np
import matplotlib.pyplot as plt

# Download file
r = requests.get('https://user-web.icecube.wisc.edu/~grbweb_public/Summary_table.txt')
with open("Summary_table.txt", 'wb') as f:
    f.write(r.content)

# Read content
data = np.loadtxt("Summary_table.txt", dtype='str',unpack='True')

# Read headers
with open("Summary_table.txt",'r') as f:
    names= np.array([n.strip().replace(" ","_") for n in f.readlines()[1].replace("#","").replace("\n","").lstrip().split('    ') if n.strip()!=''])

print(data)
#plt.plot(data[6])
#plt.hist(data[6])

[['GRB240429A' 'GRB240425A' 'GRB240422B*' ... 'GRB910424A*' 'GRB910423A*'
  'GRB910421A*']
 ['None' 'GRB240425015' 'GRB240422389' ... 'None' 'None' 'None']
 ['15:04:28' '0:21:14.965' '9:19:28.936' ... '19:43:25.064'
  '19:51:15.804' '9:14:03.800']
 ...
 ['-999' '37.3770' '42.7530' ... '3.1360' '208.5760' '5.6960']
 ['True' 'True' 'True' ... 'False' 'False' 'False']
 ['60429.62810185' '60425.01475654' '60422.38852935' ... '48370.82181787'
  '48369.82726625' '48367.38476620']]
